In [1]:
%matplotlib widget

In [2]:
import numpy as np
import ipyvolume as ipv
from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation as rot

# plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

# interactive plots
import ipywidgets as widgets

from ipywidgets import VBox, HBox, IntSlider, Checkbox, interactive_output, FloatSlider
from IPython.display import display

import libo
from libo.io import tc3

import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vs

# Setup

In [3]:
system_data_index = 17519
scan_data_scan_idx = 17507
scan_data_tcp_idx = 17518

profile_raster_width = 4
trace_raster_width = 4

# Import and process experimental data

In [4]:
# import machine data
ds_system_data_base = tc3.to_xarray(tc3.read_db(system_data_index))

# import scan data
scan_data_layer_0 = libo.io.tools.merge_scan_tcp(scan=scan_data_scan_idx, tcp=17518)

# extract system movement data
ds_movement_data = ds_system_data_base[["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz"]]

# extract welding movement data
ds_welding_movement_data = ds_movement_data.where(ds_system_data_base.trigSchweissen == 1).dropna("time")
welding_coordinates_base = ds_welding_movement_data.to_array().data[:3, :]
welding_angles_degree_base = ds_welding_movement_data.to_array().data[3:, :]

# extract scanner movement data
scanner_tcp_layer_0_coordinates_base = scan_data_layer_0.scan_tcp.sel(tcp_variable=["X", "Y", "Z"]).data
scanner_tcp_layer_0_angles_degree_base = scan_data_layer_0.scan_tcp.sel(tcp_variable=["Rx", "Ry", "Rz"]).data

# Define coordinate systems

- `cs_base` is the root coordinate system
- `cs_ref` is defined in `cs_base` -> coordinate system created from 3 robot positions
- `cs_sp` is defined in `cs_ref` -> specimen coordinate system with x pointing into the direction of the trace

All data will be transformed to the specimens coordinate system. Data which is not defined in the specimen coordinate system gets the corresponing coordinate system's suffix attached to the variable name

Important note: Measured data is not perfectly orthogonal. We use the cross product to get a z axis which is perfectly orthogonal (in floating point precision) and recalculate a perfectly orthogonal y-axis. There are certainly better ways to address this problem.

In [5]:
# measured reference coordinate system data
origin = ut.to_float_array([67.311, 1055.137, -132.694])
offset_ox = ut.to_float_array([157.345, 1054.486, -132.949])
offset_oy = ut.to_float_array([69.149, 1366.014, -131.962])

# Calculate orthogonal z axis and construct reference coordinate system from x and z axis
vec_x = offset_ox - origin
vec_y = offset_oy - origin
vec_z = np.cross(vec_x, vec_y)
cs_ref_in_base = tf.LocalCoordinateSystem.construct_from_xz_and_orientation(vec_x=vec_x, vec_z=vec_z, origin=origin)


orientation_scanner_tcp_in_flange = rot.from_euler(angles=[21.9714, -0.7410, -89.7500], seq="xyz", degrees=True).as_dcm()
orientation_torch_tcp_in_flange = rot.from_euler(angles=[-22.0474,0.5659,90.7092], seq="xyz", degrees=True).as_dcm()

# create constant coordinate systems
cs_sp_in_ref = tf.LocalCoordinateSystem(basis=tf.rotation_matrix_z(np.pi / 2), origin=[-20, -10, -8])
cs_sp_in_base = cs_sp_in_ref + cs_ref_in_base
cs_torch_tcp_in_flange = tf.LocalCoordinateSystem(basis=orientation_torch_tcp_in_flange, origin = [-49.325,-0.410,477.208])
cs_scanner_tcp_in_flange = tf.LocalCoordinateSystem(basis=orientation_scanner_tcp_in_flange, origin = [-47.949,-105.258,479.690])
cs_scan_in_scanner_tcp = tf.LocalCoordinateSystem(basis=[[1, 0, 0], [0, 1, 0], [0, 0, -1]], origin=[0, 0, 260])
cs_scanner_tcp_in_torch_tcp = cs_scanner_tcp_in_flange - cs_torch_tcp_in_flange
cs_torch_tcp_in_scanner_tcp =  cs_torch_tcp_in_flange - cs_scanner_tcp_in_flange


# create time dependent scanner coordinate systems
num_scans_layer_0 = scanner_tcp_layer_0_coordinates_base.shape[0]
scanner_tcp_layer_0_rotation_matrices_base = rot.from_euler(
    angles=scanner_tcp_layer_0_angles_degree_base, seq="xyz", degrees=True
).as_dcm()
cs_array_scan_layer_0_in_sp = []
cs_array_scanner_tcp_layer_0_in_base = []
cs_array_scanner_tcp_layer_0_in_sp = []
for i in range(num_scans_layer_0):
    orientation = scanner_tcp_layer_0_rotation_matrices_base[i]
    coordinates = scanner_tcp_layer_0_coordinates_base[i]
    cs_array_scanner_tcp_layer_0_in_base += [tf.LocalCoordinateSystem(basis=orientation, origin=coordinates)]
    cs_array_scanner_tcp_layer_0_in_sp += [cs_array_scanner_tcp_layer_0_in_base[i] - cs_ref_in_base - cs_sp_in_ref]
    cs_array_scan_layer_0_in_sp += [cs_scan_in_scanner_tcp + cs_array_scanner_tcp_layer_0_in_sp[i]]

# Define data transformations

In [6]:
def coordinates_to_specimen_from_base(coordinates):
    rotation = cs_sp_in_base.orientation.transpose()
    translation = cs_sp_in_base.location[:, np.newaxis]
    return np.matmul(rotation, coordinates - translation)


def coordinates_to_child_from_parent(coordinates, cs_child):
    rotation = cs_child.orientation.transpose()
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates - translation)


def coordinates_to_parent_from_child(coordinates, cs_child):
    rotation = cs_child.orientation
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates) + translation

# Transform data to specimen coordinate system

In [7]:
welding_coordinates_sp = coordinates_to_child_from_parent(welding_coordinates_base, cs_sp_in_base)

scanned_profiles_layer_0_sp = []
for i in range(num_scans_layer_0):
    scanned_profile_scan = scan_data_layer_0.scan_line.data[i].transpose()
    scanned_profiles_layer_0_sp += [
        coordinates_to_parent_from_child(scanned_profile_scan, cs_array_scan_layer_0_in_sp[i])
    ]

# Create theoretical geometry

In [8]:
# create points
pt_0 = [40, 8]
pt_1_1 = [np.tan(2 / 18 * np.pi) * 8, 8]
pt_1_2 = [np.tan(3 / 18 * np.pi) * 8, 8]
pt_2 = [0, 0]
pt_3 = [40, 0]

# create shapes
shape_p1_r = geo.Shape().add_line_segments([pt_0, pt_1_1, pt_2])
shape_p2_r = geo.Shape().add_line_segments([pt_0, pt_1_2, pt_2])
shape_p1_l = shape_p1_r.reflect([1, 0])
shape_p2_l = shape_p2_r.reflect([1, 0])

# create profiles
profile_1 = geo.Profile([shape_p1_l, shape_p1_r])
profile_2 = geo.Profile([shape_p2_l, shape_p2_r])

# create variable profile
variable_profile = geo.VariableProfile([profile_1, profile_2], [0, 1], [geo.linear_profile_interpolation_sbs])

# create trace
trace = geo.Trace(geo.LinearHorizontalTraceSegment(350))

# create geometry
geometry = geo.Geometry(variable_profile, trace)

# rasterize geometry
geometry_data_sp = geometry.rasterize(profile_raster_width=profile_raster_width, trace_raster_width=trace_raster_width)

# calculate triangles
triangles = Delaunay(geometry_data_sp.transpose()[:, :2]).simplices

# Plot

In [9]:
ipv.figure()
# we draw the tetrahedron

profile_idx = 1570

scanned_profiles_layer_0_sp = np.array(scanned_profiles_layer_0_sp, float)

mesh = ipv.plot_trisurf(
    geometry_data_sp[0], geometry_data_sp[1], geometry_data_sp[2], triangles=triangles, color=[0.6, 0.6, 0.6]
)
ipv.scatter(
    welding_coordinates_sp[0],
    welding_coordinates_sp[1],
    welding_coordinates_sp[2],
    size=1,
    marker="sphere",
    color=[0.3, 0.9, 0.7],
)
s = ipv.scatter(
    scanned_profiles_layer_0_sp[0::10, 0, :],
    scanned_profiles_layer_0_sp[0::10, 1, :],
    scanned_profiles_layer_0_sp[0::10, 2, :],
    size=1,
    marker="sphere",
    color="red",
)
ipv.animation_control(s)
ipv.xlim(0, 350)
ipv.ylim(-30, 30)
ipv.zlim(-10, 30)
ipv.show()

In [10]:
def create_unit_cylinder_mesh(num_segments_radial, num_segments_axial):
    num_segments_radial = np.clip(num_segments_radial, 3, None)
    num_segments_axial = np.clip(num_segments_axial, 1, None)
    delta_angle = 2 * np.pi / num_segments_radial
    delta_axial = 1 / num_segments_axial

    num_points = num_segments_radial * (num_segments_axial + 1)

    points = np.ndarray((3, num_points), float)
    for i in range(num_segments_axial + 1):
        z = i * delta_axial
        for j in range(num_segments_radial):
            angle = j * delta_angle
            x = np.sin(angle)
            y = np.cos(angle)
            points[:, i * num_segments_radial + j] = [x, y, z]

    num_triangles = num_segments_radial * num_segments_axial * 2
    triangles = np.ndarray((num_triangles, 3), int)
    for i in range(num_segments_axial):
        for j in range(num_segments_radial):
            idx_triangle = (i * num_segments_radial + j) * 2

            idx_first_point_axial_0 = i * num_segments_radial
            idx_first_point_axial_1 = idx_first_point_axial_0 + num_segments_radial
            rad_idx_0 = j
            rad_idx_1 = (j + 1) % num_segments_radial

            idx_0_0 = idx_first_point_axial_0 + rad_idx_0
            idx_0_1 = idx_first_point_axial_0 + rad_idx_1
            idx_1_0 = idx_first_point_axial_1 + rad_idx_0
            idx_1_1 = idx_first_point_axial_1 + rad_idx_1

            triangles[idx_triangle] = [idx_0_0, idx_1_0, idx_1_1]
            triangles[idx_triangle + 1] = [idx_0_0, idx_1_1, idx_0_1]

    return [points, triangles]

def create_cone_mesh(num_segments_radial):
    num_segments_radial = np.clip(num_segments_radial, 3, None)
    num_points = num_segments_radial + 1
    num_triangles = num_segments_radial
    delta_angle = 2 * np.pi / num_segments_radial
    
    points = np.ndarray((3, num_points), float)
    triangles = np.ndarray((num_triangles, 3), int)
    z = 0
    
    points[:,0] = [0, 0, 1]
    for i in range(num_segments_radial):        
        angle = i * delta_angle
        x = np.sin(angle)
        y = np.cos(angle)
        points[:, i+1] = [x, y, z]
        triangles[i] = [i+1,0,(i+1) % (num_triangles)+1]
    return [points,triangles]

create_cone_mesh(4)


    

[array([[ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  1.2246468e-16,
         -1.0000000e+00],
        [ 0.0000000e+00,  1.0000000e+00,  6.1232340e-17, -1.0000000e+00,
         -1.8369702e-16],
        [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]]),
 array([[1, 0, 2],
        [2, 0, 3],
        [3, 0, 4],
        [4, 0, 1]])]

In [11]:
scale_scanner = 15



#[vert_scanner, tri_scanner] = create_unit_cylinder_mesh(40, 1)
[vert_scanner, tri_scanner] = create_cone_mesh(40)
# vert_scanner = np.matmul(tf.rotation_matrix_y(np.pi / 2), vert_scanner)
vert_scanner = vert_scanner + np.array([0, 0, -1])[:, np.newaxis]
vert_scanner = np.matmul([[scale_scanner, 0, 0], [0, scale_scanner, 0], [0, 0, scale_scanner]], vert_scanner)


scale_torch = 15

#[vert_torch, tri_torch] = create_unit_cylinder_mesh(40, 1)
[vert_torch, tri_torch] = create_cone_mesh(40)
# vert_scanner = np.matmul(tf.rotation_matrix_y(np.pi / 2), vert_scanner)
vert_torch = vert_torch + np.array([0, 0, -1])[:, np.newaxis]
vert_torch = np.matmul([[scale_torch, 0, 0], [0, scale_torch, 0], [0, 0, scale_torch]], vert_torch)




In [12]:
out = widgets.Output(layout={"border": "2px solid black"})
# create figure inside output widget
# with out:


out2 = widgets.Output(layout={"border": "2px solid black"})
# create figure inside output widget
with out2:
    fig = plt.figure()
    gs = fig.add_gridspec(1, 1)
    ax_0 = fig.add_subplot(gs[0, 0], projection="3d")


ipvfig = ipv.figure()
layout = widgets.Layout(width="200px", height="40px")
style = {"description_width": "initial"}


# ipv.show()

play = widgets.Play(value=0, min=0, max=10, step=1, interval=500, description="Press play", disabled=False)
time_slider = IntSlider(min=0, max=10, step=1, description="time", continuous_update=True)
w1 = dict(time=time_slider)

widgets.jslink((play, "value"), (time_slider, "value"))
w = {**w1}
print(w)
# plt.show()

upper_box = HBox()
lower_box = HBox()
box = VBox([upper_box, lower_box])

ipv.plot_trisurf(
    geometry_data_sp[0], geometry_data_sp[1], geometry_data_sp[2], triangles=triangles, color=[0.6, 0.6, 0.6]
)
ipv.plot_trisurf(vert_scanner[0], vert_scanner[1], vert_scanner[2], triangles=tri_scanner, color=[1, 0, 0])
ipv.plot_trisurf(vert_torch[0], vert_torch[1], vert_torch[2], triangles=tri_torch, color=[0, 1, 0])
ipv.scatter(
    scanned_profiles_layer_0_sp[0, 0, :],
    scanned_profiles_layer_0_sp[0, 1, :],
    scanned_profiles_layer_0_sp[0, 2, :],
    size=1,
    marker="sphere",
    color="red",
)

x_min = 0
x_max = 350
y_min = -175
y_max = 175
z_min = -75
z_max = 275
ipv.xlim(x_min, x_max)
ipv.ylim(y_min, y_max)
ipv.zlim(z_min, z_max)

d_x = x_max - x_min
d_y = y_max - y_min
d_z = z_max - z_min

s_x = d_x / d_y
s_y = d_y / d_y
s_z = d_z / d_y

scale_mat = [[s_x, 0, 0], [0, s_y, 0], [0, 0, s_z]]


print(len(ipvfig.scatters))

cs_scanner_schematic = tf.LocalCoordinateSystem(origin=[0, 0, -260])


def test_func(time):
    print(time)

    time *= 100

    cs_scanner_in_sp = cs_scanner_schematic + cs_array_scanner_tcp_layer_0_in_sp[time]
    vert_scanner_upate = np.matmul(cs_scanner_in_sp.orientation, vert_scanner)
    vert_scanner_upate = np.matmul(scale_mat, vert_scanner_upate)
    vert_scanner_upate = vert_scanner_upate + cs_scanner_in_sp.location[:, np.newaxis]
    ipvfig.meshes[1].x = vert_scanner_upate[0]
    ipvfig.meshes[1].y = vert_scanner_upate[1]
    ipvfig.meshes[1].z = vert_scanner_upate[2]
    
    
    cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_array_scanner_tcp_layer_0_in_sp[time]
    vert_torch_upate = np.matmul(cs_torch_in_sp.orientation, vert_torch)
    vert_torch_upate = np.matmul(scale_mat, vert_torch_upate)
    vert_torch_upate = vert_torch_upate + cs_torch_in_sp.location[:, np.newaxis]
    ipvfig.meshes[2].x = vert_torch_upate[0]
    ipvfig.meshes[2].y = vert_torch_upate[1]
    ipvfig.meshes[2].z = vert_torch_upate[2]

    if time > 800:
        ipvfig.scatters[0].visible = False
    else:
        ipvfig.scatters.clear()
        ipv.scatter(
            scanned_profiles_layer_0_sp[time, 0, :],
            scanned_profiles_layer_0_sp[time, 1, :],
            scanned_profiles_layer_0_sp[time, 2, :],
            size=1,
            marker="sphere",
            color="red",
        )
        ipvfig.scatters[0].visible = True


test_func(7)
output = interactive_output(test_func, w)

# mesh = ipv.plot_trisurf(geometry_data_sp[0], geometry_data_sp[1], geometry_data_sp[2], triangles=triangles, color= [0.6,0.6,0.6])

upper_box.children = [ipvfig]
lower_box.children = [play, time_slider]
display(box)


# ipv.show()

{'time': IntSlider(value=0, description='time', max=10)}
1
7


In [13]:
cs_torch_tcp_in_scanner_tcp.orientation

array([[-0.99996322,  0.00628049,  0.00584105],
       [-0.00628877, -0.99997925, -0.00139949],
       [ 0.00583214, -0.00143617,  0.99998196]])